# Matrix Factorization

在存储　user-item，　计算和存储　user-user　及　item-item 相似度矩阵的时候，最大的问题就是计算和存储的复杂度为　$O(MN)$。这次依然沿用　MovieLens　[数据集](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip)。

依然沿用之前的加载数据的过程

In [1]:
import os
import pandas as pd
import numpy as np

# working directory
BASEDIR = os.getcwd()
print(BASEDIR)


dataframe = pd.read_csv(BASEDIR + '/assets/datasets/ml-latest-small/ratings.csv')

userId_dict = {}
movieId_dict = {}

userId_unique = dataframe.userId.unique()
movieId_unique = dataframe.movieId.unique()


idx = 0
for n in range(userId_unique.shape[0]):
    userId_dict[userId_unique[idx]] = idx
    idx += 1

idx = 0
for n in range(movieId_unique.shape[0]):
    movieId_dict[movieId_unique[idx]] = idx
    idx += 1

ratings = np.zeros(shape=(len(userId_dict), len(movieId_dict)))


for row in dataframe.itertuples():
    ratings[userId_dict[row.userId], movieId_dict[row.movieId]] = row.rating

/home/hailingu/Git/MLFM


矩阵分解的话，有几种做法：

- 对方阵, 可以使用　Eigen Decomposition
- 使用　Singular Value Decomposition
- 使用　Gradient Descent

在这个地方，采用　Gradient Descent，使用这个方案的原因就是因为它是之前做法的一种延续。

现在我们需要把　ratings　矩阵分解成两个矩阵，分别是　U 和　M, 其中　U 是一个　$m \times k$　的矩阵，相应的 I 就是一个　$k \times n$　的矩阵， $m, n$ 分别是　user 和　movie 的数量。

对于某一个用户　$i$　对电影　$j$ 的预计打分可以表示为 $\hat{ratings_{ij}} = u_i \cdot m_ｊ^T$，这里是向量的点乘，不是矩阵乘法，所以第二项可以写在前面，也可以写在后面。单个损失函数： $Loss_{ij} = (ratings_{ij} - \hat{ratings_{ij}})^2$。

Gradient 计算及更新：

$\begin{align*}
u_i & = u_i - \eta \cdot loss_{ij} \cdot m_j \\
m_j & = m_j - \eta \cdot loss_{ij} \cdot u_i
\end{align*}$


In [2]:
k = 150
LEARNING_RATE = 0.01

EPOCH = 1000
U = np.random.uniform(0, 1, size=(len(userId_dict), k))
M = np.random.uniform(0, 1, size=(k, len(movieId_dict)))

PRINT_STEP = EPOCH / 20
for epoch in range(EPOCH):
    hat_ratings = np.multiply(U, M)


SyntaxError: invalid syntax (<ipython-input-2-cf0e686b9e27>, line 9)